In [1]:
import contextlib
import os
import re
import sys

import linearmodels as lm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import seaborn as sns
import tabulate

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.figures.figures as figs
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh
from entropy import config

sns.set_style("whitegrid")
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [4]:
dfs, dfl = hd.read_samples(["777", "000"])
hd.inspect(dfl)

Time for read_sample    : 1.86 seconds
Time for read_sample    : 0.25 seconds
(570, 28)


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym
0,248375,2012-07-23,777,39.389999,9572 20jul 12 sacat marks spencer gemini gb cash back 30 00,NaN,spend,other_spend,0.0,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-06-25 22:55:00,natwest bank,current,2014-07-18,2017-10-23,True,2378.280029,personal,cash,groceries,groceries,u,201207
1,6755827,2013-06-21,777,7.410000,02jun a c 5781,NaN,spend,finance,0.0,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-06-25 22:55:00,natwest bank,current,2014-07-18,2017-08-14,True,2378.280029,account provider,interest charges,NaN,interest charges,u,201306


In [3]:
df

NameError: name 'df' is not defined

In [2]:
fs.ls(config.AWS_BUCKET)

['3di-project-entropy/analysis_data_777.parquet',
 '3di-project-entropy/analysis_data_XX7.parquet',
 '3di-project-entropy/analysis_data_{SAMPLE}.parquet',
 '3di-project-entropy/entropy_000.parquet',
 '3di-project-entropy/entropy_777.parquet',
 '3di-project-entropy/entropy_X77.parquet',
 '3di-project-entropy/entropy_XX7.parquet',
 '3di-project-entropy/region_lookup_table.parquet']

In [20]:
k = dfs
k["user_postcode"] = k.user_postcode.str.upper().str.replace(" ", "")
k.merge(regions, how="left", left_on="user_postcode", right_on="pcsector")

,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,user_age,entropy_sptac,pcsector,region
0,2981363,2012-11-01,7777,-800.000000,mdbremoved bac,NaN,NaN,NaN,0.0,GU149,2012-10-27,NaN,1983.0,2013-01-06,151112,2020-03-11 16:27:00,natwest bank,current,2014-07-18,2019-02-04,False,3110.219971,NaN,NaN,NaN,NaN,u,201211,3990.609131,27100.681641,38.0,2.846618,GU149,South East
1,2981500,2012-11-01,7777,11.980000,forces financial,forces financial,spend,finance,0.0,GU149,2012-10-27,NaN,1983.0,2013-01-06,151110,2020-03-11 16:27:00,natwest bank,current,2014-07-18,2017-12-04,True,22128.669922,forces financial,insurance,home insurance,home insurance,u,201211,16181.263672,27100.681641,38.0,2.846618,GU149,South East
2,2981503,2012-11-01,7777,59.779999,royal sunalliance,royal sun alliance,spend,finance,0.0,GU149,2012-10-27,NaN,1983.0,2013-01-06,151110,2020-03-11 16:27:00,natwest bank,current,2014-07-18,2017-11-13,True,22128.669922,royal sun alliance,insurance,vehicle insurance,vehicle insurance,u,201211,16181.263672,27100.681641,38.0,2.846618,GU149,South East
3,2981506,2012-11-01,7777,800.000000,mdbremoved,NaN,NaN,NaN,0.0,GU149,2012-10-27,NaN,1983.0,2013-01-06,151110,2020-03-11 16:27:00,natwest bank,current,2014-07-18,2017-10-24,True,22128.669922,non merchant mbl,NaN,other account,other account,u,201211,16181.263672,27100.681641,38.0,2.846618,GU149,South East
4,2981505,2012-11-01,7777,19.500000,5920 31oct 12 speedy chinese tak farnborough gb,NaN,NaN,NaN,0.0,GU149,2012-10-27,NaN,1983.0,2013-01-06,151110,2020-03-11 16:27:00,natwest bank,current,2014-07-18,2017-10-23,True,22128.669922,NaN,NaN,dining and drinking,dining and drinking,u,201211,16181.263672,27100.681641,38.0,2.846618,GU149,South East
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170632,768266742,2018-12-31,582777,7.990000,wh smith cheltenham,wh smith,spend,retail,1.0,GL526,2020-05-05,NaN,NaN,2020-05-05,1683911,2020-05-05 14:29:00,hsbc,current,2020-05-06,1900-01-01,True,1033.229980,wh smith,books / magazines / newspapers,NaN,books / magazines / newspapers,c,201812,3334.592285,7085.468471,NaN,2.551491,GL526,South West
170633,768268663,2018-12-31,582777,17.990000,hmv retail ltd cheltenham,hmv,spend,services,1.0,GL526,2020-05-05,NaN,NaN,2020-05-05,1683911,2020-05-05 14:29:00,hsbc,current,2020-05-06,1900-01-01,True,1033.229980,hmv,"entertainment, tv, media",NaN,"entertainment, tv, media",c,201812,3334.592285,7085.468471,NaN,2.551491,GL526,South West
170634,768261172,2018-12-31,582777,3.430000,lidl uk cheltenhamcheltenham,lidl,spend,household,1.0,GL526,2020-05-05,NaN,NaN,2020-05-05,1683911,2020-05-05 14:29:00,hsbc,current,2020-05-06,1900-01-01,True,1033.229980,lidl,"food, groceries, household",NaN,"food, groceries, household",c,201812,3334.592285,7085.468471,NaN,2.551491,GL526,South West
170635,768262440,2018-12-31,582777,32.299999,lidl uk cheltenhamcheltenham,lidl,spend,household,1.0,GL526,2020-05-05,NaN,NaN,2020-05-05,1683911,2020-05-05 14:29:00,hsbc,current,2020-05-06,1900-01-01,True,1033.229980,lidl,"food, groceries, household",NaN,"food, groceries, household",c,201812,3334.592285,7085.468471,NaN,2.551491,GL526,South West


In [12]:
regions

,pcsector,region
0,AB101,Scotland
1,AB106,Scotland
2,AB107,Scotland
3,AB115,Scotland
4,AB116,Scotland
...,...,...
11153,IM995,Isle of Man
11154,IM996,Isle of Man
11155,IM997,Isle of Man
11156,IM998,Isle of Man


Entropy analysis data

In [3]:
filepath = "/Users/fgu/tmp/en/analysis_data_XX7.parquet"
dfa = ha.read_parquet(filepath)
hd.inspect(dfa)

(163,915, 23)


obs  balance_ca  balance_sa  sa_inflows  sa_outflows  \
user_id date                                                               
57      2012-04-30   66         0.0         NaN         NaN          NaN   
        2012-05-31   75         0.0         NaN         NaN          NaN   

                    sa_net_inflows  sa_scaled_inflows  sa_scaled_outflows  \
user_id date                                                                
57      2012-04-30             NaN                NaN                 NaN   
        2012-05-31             NaN                NaN                 NaN   

                    sa_scaled_net_inflows  total_monthly_spend  \
user_id date                                                     
57      2012-04-30                    NaN             6.784095   
        2012-05-31                    NaN             6.837290   

                    tag_spend_household  tag_spend_hobbies  tag_spend_retail  \
user_id date                                                                   
57      2012-04-30             0.400869           0.011305           0.00937   
        2012-05-31             0.469773           0.010719           0.00000   

                    tag_spend_services  tag_spend_other_spend  \
user_id date                                                    
57      2012-04-30            0.245824               0.067898   
        2012-05-31            0.317825              -0.014239   

                    tag_spend_finance  tag_spend_travel  \
user_id date                                              
57      2012-04-30           0.000837          0.228929   
        2012-05-31           0.001105          0.148343   

                    tag_spend_communication  tag_spend_motor  entropy_sptac  \
user_id date                                                                  
57      2012-04-30                 0.034967              0.0       2.425209   
        2012-05-31                 0.066473              0.0       2.363752   

                    log_income  user_female   age  
user_id date                                       
57      2012-04-30   10.023162          0.0  25.0  
        2012-05-31   10.023162          0.0  25.0

In [6]:
fs.ls("3di-data-mdb/raw")

['3di-data-mdb/raw/',
 '3di-data-mdb/raw/20200630_UserLoginsForNeedham.csv',
 '3di-data-mdb/raw/mdb_000.csv',
 '3di-data-mdb/raw/mdb_000.parquet',
 '3di-data-mdb/raw/mdb_777.csv',
 '3di-data-mdb/raw/mdb_777.parquet',
 '3di-data-mdb/raw/mdb_X77.csv',
 '3di-data-mdb/raw/mdb_X77.parquet',
 '3di-data-mdb/raw/mdb_XX7.csv',
 '3di-data-mdb/raw/mdb_XX7.parquet',
 '3di-data-mdb/raw/mdb_costa.csv']

### MDB user login data

In [93]:
@hh.timer
def read_logins(nrows=None):
    fp = "s3://3di-data-mdb/raw/20200630_UserLoginsForNeedham.csv"
    return ha.read_csv(fp, nrows=nrows, names=["user_id", "date"], parse_dates=["date"])


login_data = read_logins()
hd.inspect(login_data)

Time for read_logins    : 37.96 seconds
(7,521,416, 2)


,user_id,date
0,1,2015-07-23 14:28:00
1,1,2015-07-27 05:43:00


Proportion of users in dataset with login info

In [94]:
len(set(dfl.user_id) & set(login_data.user_id)) / dfl.user_id.nunique()

0.8884974458340673

Merge login data into transaction data

In [132]:
def make_daily_logins(login_data):
    login_data = login_data.copy()
    login_data["date"] = login_data.date.dt.round("d")
    login_data["logins"] = 1
    return login_data.groupby(["user_id", "date"]).logins.sum().reset_index()


def merge_logins(txn_data, login_data):
    txn_data = txn_data.copy()
    login_data = login_data.copy()
    merged = txn_data.merge(
        login_data, on=["user_id", "date"], how="left", validate="m:1"
    )
    merged["logins"] = merged.logins.fillna(0)
    return merged


def drop_pre_registration_data(df):
    """Drop all transactions from before a user registered with MDB.

    MDB receives up to three years of data after a user signs up. As
    a user can obviously not log in to their MDB account during that
    time, we drop these transactions.
    """
    return df[df.date >= df.user_registration_date]


txn_data = dfl
daily_logins = make_daily_logins(logins)
post_registration_txns = drop_pre_registration_data(txn_data)
df = merge_logins(post_registration_txns, daily_logins)
hd.inspect(df)

(9,539,059, 32)


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,entropy_sptac,logins
0,1088993,2012-04-02,57,23.74,1982 01apr 12 the broadway fruiterer london gb,NaN,NaN,NaN,0.0,n8 9,2010-05-13,10k to 20k,1987.0,2016-10-30,273998,2015-08-15 21:37:00,natwest bank,current,2014-07-18,2017-10-23,True,NaN,NaN,NaN,NaN,NaN,u,201204,NaN,22542.600586,2.425209,0.0
1,1088994,2012-04-02,57,25.68,1982 01apr 12 waitrose 835 crouch end gb,waitrose,spend,household,0.0,n8 9,2010-05-13,10k to 20k,1987.0,2016-10-30,273998,2015-08-15 21:37:00,natwest bank,current,2014-07-18,2017-08-15,True,NaN,waitrose,"food, groceries, household",NaN,supermarket,u,201204,NaN,22542.600586,2.425209,0.0


In [145]:
def user_logins(df, freq="m"):
    period = pd.Grouper(key="date", freq=freq)
    return (
        df.groupby(["user_id", period])
        .logins.sum()
        .groupby(["user_id"])
        .agg(["mean", "median", "std", "size"])
    )


user_logins_data = user_logins(df)

In [146]:
hd.inspect(user_logins_data)

(5,204, 4)


,mean,median,std,size
user_id,,,,
57,8.160000,0.0,13.754500,100
107,11.605263,1.0,25.055181,76


In [147]:
user_logins_data.describe()

,mean,median,std,size
count,5204.000000,5204.000000,4992.000000,5204.000000
mean,6.809477,4.682840,7.701427,18.873751
std,13.708230,13.135486,11.108252,19.204074
min,0.000000,0.000000,0.000000,1.000000
25%,0.206643,0.000000,0.707107,5.000000
50%,1.838542,0.000000,3.714036,11.000000
75%,6.893939,3.000000,10.035591,25.000000
max,180.625000,181.500000,123.036580,100.000000
